## DKRZ NCL notebook example
<table align="left">
<tr><td>Title:</td><td>Masking</td></tr>
<tr><td>Description</td><td>Demonstrate how to compute a mask file and use it to mask a variable</td></tr>
<tr><td>20.07.18</td><td>kmf</td></tr>
</table>

This example will show you how to mask your data when you do not have a mask file. In this case the CDO (Climate Data Operators) are used to compute the mask on the desired grid. *After generating the mask file comment the line 'system(cmd)' at the top of this script.*</br></br>

The ESFM_regridding.ncl library must be loaded on top of the script only when your NCL is older than the version 6.5.0 (ncl -V).


In [29]:
load "$NCARG_ROOT/lib/ncarg/nclscripts/esmf/ESMF_regridding.ncl" ;-- since v6.5.0 already loaded

The name of the data file.

In [30]:
filename = "$HOME/data/CMIP5/atmos/tas_Amon_MPI-ESM-LR_historical_r1i1p1_1980.nc"

=
>


Compute the mask file using CDO's topo operator, remap it to the desired grid (here a 1x1 degree grid
= r360x180), and save it as mask.nc file. </br>
**After generating the mask file once comment the line 'system(cmd)' at the top of this script with *;* .**

In [31]:
cmd = "cdo -f nc -chname,topo,mask -expr,'topo=((topo>0.0))?1.0:0.0' -remapycon,r360x180 -topo mask.nc"
system(cmd)

Open file and read the variable.

In [32]:
f   = addfile(filename,"r")
tas = f->tas(0,:,:)

Set some ESMF resources.

In [33]:
  Opt                     =  True
  Opt@InterpMethod        = "bilinear"
  Opt@ForceOverwrite      =  True
  Opt@DstGridType         = "1x1"                       ;-- Destination grid
  Opt@DstLLCorner         =  (/-89.75d,   0.00d /)
  Opt@DstURCorner         =  (/ 89.75d, 359.75d /)

Do the regridding of the data to the desired grid (1x1 degrees)

In [34]:
tas_regrid = ESMF_regrid(tas,Opt)

Open mask file and read mask variable.

In [35]:
m  = addfile("mask.nc","r")
ma = m->mask

Mask the ocean.

In [36]:
land_only = tas_regrid                                ;-- copy variable to keep the metadata
land_only = mask(tas_regrid,ma,1)                     ;-- retrieve tas_regrid where ma=1

=
>=
>=
>


Open a workstation. Save graphic output to PNG file.

In [37]:
wks_type = "png"
wks_type@wkWidth  = 400
wks_type@wkHeight = 400
wks = gsn_open_wks(wks_type,"plot_notebook_masking")

Set plot resources: maximize plot, use contour fill, no contour lines, and raster fill mode

In [38]:
res =  True
res@gsnMaximize   =  True

res@cnFillOn      =  True
res@cnLinesOn     =  False
res@cnFillMode    = "RasterFill"

First plot: Create the contour plot of the regridded variable tas. **To get rid of the white space around the plot use ImageMagicks *convert* program.**

In [39]:
plot_data = gsn_csm_contour_map(wks,tas_regrid,res)

In [40]:
system("convert -alpha off -background white -density 300 -trim plot_notebook_masking.000001.png tmp1.png; mv tmp1.png plot_notebook_masking.000001.png") 

![notebook masking](plot_notebook_masking.000001.png)

Second plot: Create the plot of the mask file which contains only 0 and 1.

In [41]:
plot_mask_file = gsn_csm_contour_map(wks,ma,res)

system("convert -alpha off -background white -density 300 -trim plot_notebook_masking.000002.png tmp2.png; mv tmp2.png plot_notebook_masking.000002.png") 

![notebook masking](plot_notebook_masking.000002.png)

Third Plot: Create the plot of the masked data tas_regrid.

In [42]:
plot_mask_data = gsn_csm_contour_map(wks,land_only,res)

system("convert -alpha off -background white -density 300 -trim plot_notebook_masking.000003.png tmp3.png; mv tmp3.png plot_notebook_masking.000003.png") 

![notebook masking](plot_notebook_masking.000003.png)